In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging
import argparse
import subprocess
from imp import reload
from tensorflowonspark import TFCluster

Waiting for a Spark session to start...

Waiting for a Spark session to start...

In [2]:
sc.addPyFile('TensorFlowOnSpark/examples/mnist/spark/mnist_dist.py')

In [3]:
import mnist_dist

In [4]:
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

In [5]:
parser = argparse.ArgumentParser()
parser.add_argument("--epochs", help="number of epochs", type=int, default=1)
parser.add_argument("--images", help="HDFS path to MNIST images in parallelized format")
parser.add_argument("--labels", help="HDFS path to MNIST labels in parallelized format")
parser.add_argument("--format", help="example format", choices=["csv","pickle","tfr"], default="csv")
parser.add_argument("--model", help="HDFS path to save/load model during train/test", default="mnist/mnist_csv_model")
parser.add_argument("--readers", help="number of reader/enqueue threads", type=int, default=1)
parser.add_argument("--steps", help="maximum number of steps", type=int, default=500)
parser.add_argument("--batch_size", help="number of examples per batch", type=int, default=100)
parser.add_argument("--mode", help="train|inference", default="train")
parser.add_argument("--rdma", help="use rdma connection", default=False)
num_executors = 5

In [6]:
#remove existing models if any
subprocess.call(["hdfs", "dfs", "-rm", "-R", "-skipTrash", "mnist/mnist_csv_model"])

0

In [7]:
#verify training images
train_images_files = "mnist/csv/train/images"
print(subprocess.check_output(["hdfs", "dfs", "-ls", "-R", train_images_files]))

b'-rw-r--r--   3 nobody supergroup          0 2018-06-10 20:05 mnist/csv/train/images/_SUCCESS\n-rw-r--r--   3 nobody supergroup    9338236 2018-06-10 20:05 mnist/csv/train/images/part-00000\n-rw-r--r--   3 nobody supergroup   11231804 2018-06-10 20:05 mnist/csv/train/images/part-00001\n-rw-r--r--   3 nobody supergroup   11214784 2018-06-10 20:05 mnist/csv/train/images/part-00002\n-rw-r--r--   3 nobody supergroup   11226100 2018-06-10 20:05 mnist/csv/train/images/part-00003\n-rw-r--r--   3 nobody supergroup   11212767 2018-06-10 20:05 mnist/csv/train/images/part-00004\n-rw-r--r--   3 nobody supergroup   11173834 2018-06-10 20:05 mnist/csv/train/images/part-00005\n-rw-r--r--   3 nobody supergroup   11214285 2018-06-10 20:05 mnist/csv/train/images/part-00006\n-rw-r--r--   3 nobody supergroup   11201024 2018-06-10 20:05 mnist/csv/train/images/part-00007\n-rw-r--r--   3 nobody supergroup   11194141 2018-06-10 20:05 mnist/csv/train/images/part-00008\n-rw-r--r--   3 nobody supergroup   10449

In [8]:
#verify training labels
train_labels_files = "mnist/csv/train/labels"
print(subprocess.check_output(["hdfs", "dfs", "-ls", "-R", train_labels_files]))

b'-rw-r--r--   3 nobody supergroup          0 2018-06-10 20:05 mnist/csv/train/labels/_SUCCESS\n-rw-r--r--   3 nobody supergroup     204800 2018-06-10 20:05 mnist/csv/train/labels/part-00000\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00001\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00002\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00003\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00004\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00005\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00006\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00007\n-rw-r--r--   3 nobody supergroup     245760 2018-06-10 20:05 mnist/csv/train/labels/part-00008\n-rw-r--r--   3 nobody supergroup     229

In [9]:
#Parse arguments for training
args = parser.parse_args(['--mode', 'train', '--steps', '3000', '--epochs', '5',
                          '--images', train_images_files, 
                          '--labels', train_labels_files])

In [10]:
#start the cluster for training
cluster = TFCluster.run(sc, mnist_dist.map_fun, args, num_executors, 1, True, TFCluster.InputMode.SPARK)

08:40:26 INFO:Reserving TFSparkNodes w/ TensorBoard
08:40:26 INFO:cluster_template: {'ps': range(0, 1), 'worker': range(1, 5)}
08:40:26 INFO:listening for reservations at ('10.0.3.39', 35517)
08:40:26 INFO:Starting TensorFlow on executors
08:40:26 INFO:Waiting for TFSparkNodes to start
08:40:26 INFO:waiting for 5 reservations
08:40:27 INFO:waiting for 5 reservations
08:40:28 INFO:waiting for 5 reservations
08:40:29 INFO:waiting for 5 reservations
08:40:30 INFO:waiting for 3 reservations
08:40:31 INFO:all reservations completed
08:40:31 INFO:All TFSparkNodes started
08:40:31 INFO:{'executor_id': 2, 'host': '10.0.0.69', 'job_name': 'worker', 'task_index': 1, 'port': 34437, 'tb_pid': 0, 'tb_port': 0, 'addr': '/tmp/pymp-yfhjek51/listener-0vsjzptz', 'authkey': b'\xef\xfc\xf5\xc9\x18\x9dL\x02\x96yv\xa70\x85\xe4\xab'}
08:40:31 INFO:{'executor_id': 4, 'host': '10.0.2.241', 'job_name': 'worker', 'task_index': 3, 'port': 42945, 'tb_pid': 0, 'tb_port': 0, 'addr': '/tmp/pymp-cxco13ug/listener-hq8z

In [ ]:
#Feed data via Spark RDD
images = sc.textFile(args.images).map(lambda ln: [int(x) for x in ln.split(',')])
labels = sc.textFile(args.labels).map(lambda ln: [float(x) for x in ln.split(',')])
dataRDD = images.zip(labels)
cluster.train(dataRDD, args.epochs)

08:40:33 INFO:Feeding training data


In [ ]:
cluster.shutdown()

In [ ]:
print(subprocess.check_output(["hdfs", "dfs", "-ls", "mnist/mnist_csv_model"]))

In [ ]:
#verify test images
test_images_files = "mnist/csv/test/images"
print(subprocess.check_output(["hdfs", "dfs", "-ls", test_images_files]))

In [ ]:
#verify test labels
test_labels_files = "mnist/csv/test/labels"
print(subprocess.check_output(["hdfs", "dfs", "-ls", test_labels_files]))

In [ ]:
#Parse arguments for inference
args = parser.parse_args(['--mode', 'inference', 
                          '--images', test_images_files, 
                          '--labels', test_labels_files])

In [ ]:
#Start the cluster for inference
cluster = TFCluster.run(sc, mnist_dist.map_fun, args, num_executors, 1, False, TFCluster.InputMode.SPARK)

In [ ]:
#prepare data as Spark RDD
images = sc.textFile(args.images).map(lambda ln: [int(x) for x in ln.split(',')])
labels = sc.textFile(args.labels).map(lambda ln: [float(x) for x in ln.split(',')])
dataRDD = images.zip(labels)
#feed data for inference
prediction_results = cluster.inference(dataRDD)
prediction_results.take(20)

In [ ]:
cluster.shutdown()